# Create your Index for Similarity Search

![Converting our Plain Text Docs into chunked docs in an opensearch index](../img/txt-doc-to-pg-docs.png)

In order to ingest our transcriptions we need to prepare an opensearch index to store our data.

In this workshop, we're ingesting ONLY [our transcription example](../transcripts/transcription_example.txt) but our opensearch index will have hundreds of documents and our final RAG Application will have tens of thousands of documents.

---

🔍 Let's examine the metadata of our document

```yaml
description: "Do you have a grip on productivity? Are you worried that external factors could disrupt what you’re doing at any second? Time to put things in a VICE!"
pub_date: "March 10th, 2022"
title: "18: Putting External Factors in a VICE Grip \U0001F5DC"
url: https://relay.fm/conduit/18
```

This information along with our `content` needs to be mapped out into an index.

While all of the metadata is a string we want to setup our metadata to fit our needs which means `pub_date` should be a `date` value.

Let's start out by importing our environment variables and loading our imports. Then we'll establish our connection with our OpenSearch®️ service.

In [ ]:
import os
import psycopg
from dotenv import load_dotenv

load_dotenv()

# Connection parameters
_CONNECTION_STRING = os.getenv("AIVEN_POSTGRES_SERVICE_URI")
conn = psycopg.connect(_CONNECTION_STRING)
print(conn)

Next, we need to enable the PGVector Extension and define our tables. 

In [ ]:
with conn.cursor() as cur:
    # Check if vector extension exists
    get_vector = "SELECT EXISTS(SELECT 1 FROM pg_extension WHERE extname = 'vector');"
    cur.execute(get_vector)
    extension_exists = cur.fetchone()[0]

    if not extension_exists:
        cur.execute("CREATE EXTENSION vector;")
        cur.execute(get_vector)
    conn.commit()

Let's create a function that will create our table and the fields for it.

We'll use two tables – one for transcriptions and the other for quotes and their embeddings.

In [ ]:
def create_table(cursor, table: str, fields_str: str) -> None:
    cursor.execute(
        f"SELECT EXISTS(SELECT 1 FROM information_schema.tables WHERE table_name = '{table}');"
    )
    table_exists = cursor.fetchone()[0]

    if not table_exists:
        cursor.execute(f"CREATE TABLE {table} ({fields_str});")
        logging.info("%s table created successfully." % table)
    else:
        logging.debug("%s table already exists." % table)

# Run the function for each table

try:
    # Establish the connection
    with conn.cursor() as cur:
        # Check if vector extension exists
        transcription_fields = (
                "title TEXT PRIMARY KEY",
                "content TEXT",
                "meta JSONB",
        )
        
        create_table(cur, "transcriptions", ",".join(transcription_fields))

        quote_chunks_fields = (
            "id SERIAL PRIMARY KEY",
            "content TEXT",
            "embedding vector(768)",
            "transcription_title TEXT REFERENCES transcriptions(title)",  # Foreign key to transcriptions table
        )

        create_table(cur, "quotes", ",".join(quote_chunks_fields))
        conn.commit()

except Exception as exc:
    print(f"{exc.__class__.__name__}: {exc}")


In [ ]:
with conn.cursor() as cur:
    cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public' AND table_type='BASE TABLE'")
    tables = cur.fetchall()
    print(tables)
    

In this notebook we created our PostgreSQL Tables. We looked at the metadata and made sure that the values matched.

In the next notebook we'll split our documents to fit our vectorization model and generate embeddings.

Move onto the [next notebook](2-chunk-segment-ingest.ipynb) or push the button below

[![Chunk and Ingest your Data](https://img.shields.io/badge/2-Chunk%20and%20Ingest%20Your%20Docs-153a5a?style=for-the-badge&labelColor=ec6147)](2-chunk-segment-ingest.ipynb)
